# 필요한 모듈 import

In [1]:
import pandas as pd
import numpy as np
import re
import json
import os
import sentencepiece as spm
from torch.utils.data import Dataset, DataLoader
from torch.nn import Transformer
from torch import nn
import torch
import math

# 데이터 전처리(불필요한 데이터 제거) 

In [22]:
dir_path = "/home/dilab05/work_directory/adj/NLP_Project/Training/금융보험/민원(콜센터) 질의응답_금융보험_잔고 및 거래내역_Training.json"
with open(dir_path, 'r') as file:
    li = []
    data = json.load(file)
    for i in range(len(data)):
        del(data[i]['도메인'])
        del(data[i]['카테고리'])
        del(data[i]['대화셋일련번호'])
        del(data[i]['화자'])
        del(data[i]['문장번호'])
        del(data[i]['고객의도'])
        del(data[i]['상담사의도'])
        del(data[i]['QA'])
        del(data[i]['개체명 '])
        del(data[i]['용어사전'])
        del(data[i]['지식베이스'])
        del(data[i]['상담사질문(요청)'])
        del(data[i]['고객답변'])
    for i in range(len(data)):    # 84031
        if (data[i]['고객질문(요청)'] == '') and (data[i]['상담사답변'] == ''):
            del data[i]['고객질문(요청)']
            del data[i]['상담사답변']
        else:
            if data[i]['고객질문(요청)'] == '':
                del data[i]['고객질문(요청)']
            elif data[i]['상담사답변'] == '':
                del data[i]['상담사답변']
            else:
                del data[i]['고객질문(요청)']
                del data[i]['상담사답변']
            li.append(data[i])
            

# 데이터 전처리(고객질문 다음 상담사 답변이 오는 것만 리스트에 저장)

In [24]:
idxa = []
idxq = []
for i in range(len(li)):
    if '고객질문(요청)' in li[i] and '상담사답변' in li[i+1]:
        idxq.append(i)

# 리스트에 매핑시키기 위해 넘파이 배열로 변환

In [26]:
li = np.array(li)
idxa = np.array(idxa)
idxq = np.array(idxq)

In [27]:
print(li)
print(idxa)
print(idxq)

[{'고객질문(요청)': '집과 직장에서 인터넷 뱅킹을 이용가능하나요?'}
 {'상담사답변': '가능합니다. 공인인증서를 복사해서 사용할 수 있습니다.'}
 {'고객질문(요청)': '인증서 복사는 어떻게 하죠?'} ...
 {'상담사답변': '네 000님 이름으로 30만원 입금확인 됩니다.'} {'고객질문(요청)': '12명 총 입금액이 얼마인가요?'}
 {'상담사답변': '350만원 이십니다.'}]
[]
[    0     2     4 ... 48776 48779 48781]


# 고객질문과 상담사 답변 데이터 매핑

In [28]:
q = li[idxq]
a = li[idxq+1]

#question = []
#answer = []
for i in range(len(q)):
    question.append(q[i]['고객질문(요청)'])
for i in range(len(q)):
    answer.append(a[i]['상담사답변'])

# 넘파이 배열로 저장

In [5]:
np.savez_compressed('./data', question=question, answer=answer)

In [12]:
question = np.load('./data.npz')['question']
answer = np.load('./data.npz')['answer']

# 판다스 데이터 프레임 형식으로 저장

In [13]:
train_data = pd.DataFrame([ x for x in zip(question, answer)], columns = ['Q','A'])
train_data

,Q,A
0,인터넷뱅킹 로그인이 안돼요?,인터넷뱅킹 로그인시 비밀번호 5회이상 틀리시면 로그인이 불가능합니다
1,오늘은 3번밖에 안 했는데요?,비밀번호 오류횟수가 누적 5회상이면 이용하실 수 없습니다.
2,다시 인터넷뱅킹에 로그인 하려면 어떻게 하면 되나요?,은행을 방문하셔서 비밀번호 오류를 해제하고 새 비밀번호를 등록하면 다시 이용하실 수...
3,그것만 하면 되나요?,"은행 출금계좌번호와 비밀번호, 보안매체. 본인인증이 필요합니다."
4,본인인증하기가 어려우면 어떻게 하나요?,네 번거로우시더라도 은행을 방문하셔야 합니다.
...,...,...
95560,매수주문 했던건 거래가 안됐나요?,네 매수가가 너무 낮아서 주문에 실패했습니다.
95561,주문은 취소 된거죠?,장 마감까지 매매되지 않으면 취소 됩니다.
95562,통장 거래내역 알아 볼 수 있나요?,본인확인후 안내드리겠습니다.
95563,000이름으로 입금된 건이 있나요?,네 000님 이름으로 30만원 입금확인 됩니다.


In [14]:
q_null = train_data[train_data['Q'] == ' '].index
train_data = train_data.drop(q_null)
a_null = train_data[train_data['A'] == ' '].index
train_data = train_data.drop(a_null)

# 불필요한 특수문자 제거

In [5]:
questions = []
for sentence in train_data['Q']:
	# 구두점에 대해서 띄어쓰기
    # ex) 12시 땡! -> 12시 땡 !
    sentence = re.sub(r"([?.!,~])", r"", sentence)
    sentence = sentence.strip()
    questions.append(sentence)

answers = []
for sentence in train_data['A']:
    sentence = re.sub(r"([?.!,~])", r"", sentence)
    sentence = sentence.strip()
    answers.append(sentence)

# 질문과 답변을 txt파일로 저장

In [6]:
with open('data.txt', 'w', encoding='utf8') as f:
    f.write('\n'.join(questions))
    f.write('\n'.join(answers))

# sentencepiece를 활용하여 8000개의 vocab size를 가지고 사용자 지정 토큰 7개를 추가로 가지고 있는sentence piece를 학습

In [6]:
corpus = "data.txt"
prefix = "chatbot"
vocab_size = 8000
spm.SentencePieceTrainer.train(
    f"--input={corpus} --model_prefix={prefix} --vocab_size={vocab_size + 7}" + 
    " --model_type=bpe" +
    " --max_sentence_length=999999" + # 문장 최대 길이
    " --pad_id=0 --pad_piece=[PAD]" + # pad (0)
    " --unk_id=1 --unk_piece=[UNK]" + # unknown (1)
    " --bos_id=2 --bos_piece=[BOS]" + # begin of sequence (2)
    " --eos_id=3 --eos_piece=[EOS]" + # end of sequence (3)
    " --user_defined_symbols=[SEP],[CLS],[MASK]") # 사용자 정의 토큰

sentencepiece_trainer.cc(177) LOG(INFO) Running command: --input=data.txt --model_prefix=chatbot --vocab_size=8007 --model_type=bpe --max_sentence_length=999999 --pad_id=0 --pad_piece=[PAD] --unk_id=1 --unk_piece=[UNK] --bos_id=2 --bos_piece=[BOS] --eos_id=3 --eos_piece=[EOS] --user_defined_symbols=[SEP],[CLS],[MASK]
sentencepiece_trainer.cc(77) LOG(INFO) Starts training with : 
trainer_spec {
  input: data.txt
  input_format: 
  model_prefix: chatbot
  model_type: BPE
  vocab_size: 8007
  self_test_sample_size: 0
  character_coverage: 0.9995
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 999999
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  user_defined_symbols: [SEP]
  user_defined_symbols: [CLS]
  user_defined_s

# 토큰화 테스트

In [7]:
vocab_file = "chatbot.model"
vocab = spm.SentencePieceProcessor()
vocab.load(vocab_file)
line = "무엇을 도와드릴까요?"
pieces = vocab.encode_as_pieces(line)
ids = vocab.encode_as_ids(line)


print(line)
print(pieces)
print(ids)

무엇을 도와드릴까요?
['▁무엇을', '▁도와드릴까요', '?']
[1538, 5673, 1]


ize=2420 all=123311 active=6418 piece=▁지불
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=152 size=2440 all=123522 active=6629 piece=▁1000
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=150 size=2460 all=123868 active=6975 piece=▁요양
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=148 size=2480 all=124203 active=7310 piece=입원
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=147 size=2500 all=124701 active=7808 piece=▁이것도
bpe_model_trainer.cc(167) LOG(INFO) Updating active symbols. max_freq=147 min_freq=33
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=145 size=2520 all=124935 active=6470 piece=▁대물
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=144 size=2540 all=125138 active=6673 piece=▁신체
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=143 size=2560 all=125481 active=7016 piece=▁발생할
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=141 size=2580 all=125795 active=7330 piece=▁상이
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=140 size=2600 all=126184 active=7719 piece=다구요
bpe_model_trai

# 학습된 sentence piece를 이용하여 주어진 문장을 정수로 인코딩하는 함수를 선언. 
# 문장의 처음과 끝에는 sentence piece를 학습 시킬 때 따로 선언했던 START_TOKEN과 END_TOKEN의 index를 넣음

In [8]:
MAX_LENGTH = 40

START_TOKEN = [2]
END_TOKEN = [3]

# 토큰화 / 정수 인코딩 / 시작 토큰과 종료 토큰 추가 / 패딩
def tokenize_and_filter(inputs, outputs):
    tokenized_inputs, tokenized_outputs = [], []

    for (sentence1, sentence2) in zip(inputs, outputs):
    # encode(토큰화 + 정수 인코딩), 시작 토큰과 종료 토큰 추가
        zeros1 = np.zeros(MAX_LENGTH, dtype=int)
        zeros2 = np.zeros(MAX_LENGTH, dtype=int)
        sentence1 = START_TOKEN + vocab.encode_as_ids(sentence1) + END_TOKEN
        zeros1[:len(sentence1)] = sentence1[:MAX_LENGTH]

        sentence2 = START_TOKEN + vocab.encode_as_ids(sentence2) + END_TOKEN
        zeros2[:len(sentence2)] = sentence2[:MAX_LENGTH]

        tokenized_inputs.append(zeros1)
        tokenized_outputs.append(zeros2)
    return tokenized_inputs, tokenized_outputs

# 인코딩 테스트

In [9]:
questions_encode, answers_encode = tokenize_and_filter(questions, answers)

In [10]:
print(questions_encode[1])
print(answers_encode[1])

[   2 1310 7119  135 7241 1722   50 5180    3    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0]
[   2  578 6557 1412 4431  193 7191 7130  876 1733   14  300    3    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0]


# Batch 학습을 위해 dataset, dataloader를 생성
# 첫번째 값은 주어진 질문, 두번째 값은 디코더의 입력으로 마지막 토큰값이 제거된 대답, 마지막 값은 첫 토큰값이 제거된 결과

In [11]:
class SequenceDataset(Dataset):
    def __init__(self, questions, answers):
        questions = np.array(questions)
        answers = np.array(answers)
        self.inputs = questions
        self.dec_inputs = answers[:,:-1]
        self.outputs = answers[:,1:]
        self.length = len(questions)
    
    def __getitem__(self,idx):
        return (self.inputs[idx], self.dec_inputs[idx], self.outputs[idx])

    def __len__(self):
        return self.length

BATCH_SIZE = 64
dataset = SequenceDataset(questions_encode, answers_encode)
dataloader = DataLoader(dataset, shuffle=True, batch_size=BATCH_SIZE)

# Transformer 모델 선언(positional encoding과 Embedding이 포함)

In [12]:
class TFModel(nn.Module):
    def __init__(self, ntoken, ninp, nhead, nhid, nlayers, dropout=0.5):
        super(TFModel, self).__init__()
        self.transformer = Transformer(ninp, nhead, dim_feedforward=nhid, num_encoder_layers=nlayers, num_decoder_layers=nlayers,dropout=dropout)
        self.pos_encoder = PositionalEncoding(ninp, dropout)
        self.encoder = nn.Embedding(ntoken, ninp)

        self.pos_encoder_d = PositionalEncoding(ninp, dropout)
        self.encoder_d = nn.Embedding(ntoken, ninp)

        self.ninp = ninp
        self.ntoken = ntoken

        self.linear = nn.Linear(ninp, ntoken)
        self.init_weights()

    def generate_square_subsequent_mask(self, sz):
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask

    def init_weights(self):
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, src, tgt, srcmask, tgtmask, srcpadmask, tgtpadmask):
        src = self.encoder(src) * math.sqrt(self.ninp)
        src = self.pos_encoder(src)

        tgt = self.encoder_d(tgt) * math.sqrt(self.ninp)
        tgt = self.pos_encoder_d(tgt)


        output = self.transformer(src.transpose(0,1), tgt.transpose(0,1), srcmask, tgtmask, src_key_padding_mask=srcpadmask, tgt_key_padding_mask=tgtpadmask)
        output = self.linear(output)
        return output

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

def gen_attention_mask(x):
    mask = torch.eq(x, 0)
    return mask

# 훈련 진행

In [13]:
device = torch.device("cuda", index=1)

lr = 1e-4
model = TFModel(vocab_size+7, 256, 8, 512, 2, 0.1).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [36]:
epoch = 861
best_loss = 2000.0

from tqdm import tqdm

model.load_state_dict(torch.load("chatbot.pth"))
model.train()
for i in range(epoch):
    batchloss = 0.0
    progress = tqdm(dataloader)
    for (inputs, dec_inputs, outputs) in progress:
        optimizer.zero_grad()
        src_mask = model.generate_square_subsequent_mask(MAX_LENGTH).to(device)
        src_padding_mask = gen_attention_mask(inputs).to(device)
        tgt_mask = model.generate_square_subsequent_mask(MAX_LENGTH-1).to(device)
        tgt_padding_mask = gen_attention_mask(dec_inputs).to(device)

        result = model(inputs.to(device), dec_inputs.to(device), src_mask, tgt_mask, src_padding_mask,tgt_padding_mask)
        loss = criterion(result.permute(1,2,0), outputs.to(device).long())
        progress.set_description("{:0.3f}".format(loss))
        loss.backward()
        optimizer.step()
        batchloss += loss
    print(loss)
    print(batchloss)
    if best_loss > batchloss:
        print(f"Best loss: {best_loss}, Loss: {batchloss}")
        best_loss = batchloss
        torch.save(model.state_dict(), "chatbot.pth")
            
    print("epoch:",i+1,"|","loss:",batchloss.cpu().item() / len(dataloader))

0.188: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1490/1490 [00:35<00:00, 42.57it/s]


tensor(0.1875, device='cuda:1', grad_fn=<NllLoss2DBackward>)
tensor(528.2755, device='cuda:1', grad_fn=<AddBackward0>)
Best loss: 2000.0, Loss: 528.2755126953125
epoch: 1 | loss: 0.35454732395658556


0.362: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1490/1490 [00:35<00:00, 41.71it/s]


tensor(0.3623, device='cuda:1', grad_fn=<NllLoss2DBackward>)
tensor(526.2171, device='cuda:1', grad_fn=<AddBackward0>)
Best loss: 528.2755126953125, Loss: 526.2171020507812
epoch: 2 | loss: 0.3531658403025377


0.287: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1490/1490 [00:35<00:00, 41.96it/s]


...

tensor(0.2437, device='cuda:1', grad_fn=<NllLoss2DBackward>)
tensor(187.1381, device='cuda:1', grad_fn=<AddBackward0>)
epoch: 859 | loss: 0.12559604516765416


0.079: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1490/1490 [00:36<00:00, 41.00it/s]


tensor(0.0791, device='cuda:1', grad_fn=<NllLoss2DBackward>)
tensor(186.9366, device='cuda:1', grad_fn=<AddBackward0>)
epoch: 860 | loss: 0.12546082567048553


0.128: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1490/1490 [00:36<00:00, 41.14it/s]


tensor(0.1284, device='cuda:1', grad_fn=<NllLoss2DBackward>)
tensor(186.3490, device='cuda:1', grad_fn=<AddBackward0>)
Best loss: 186.89195251464844, Loss: 186.34898376464844
epoch: 861 | loss: 0.12506643205681103


# 평가

In [37]:
def preprocess_sentence(sentence):
    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = sentence.strip()
    return sentence

def evaluate(sentence):
    sentence = preprocess_sentence(sentence)
    input = torch.tensor([START_TOKEN + vocab.encode_as_ids(sentence) + END_TOKEN]).to(device)
    output = torch.tensor([START_TOKEN]).to(device)

    # 디코더의 예측 시작
    model.eval()
    for i in range(MAX_LENGTH):
        src_mask = model.generate_square_subsequent_mask(input.shape[1]).to(device)
        tgt_mask = model.generate_square_subsequent_mask(output.shape[1]).to(device)

        src_padding_mask = gen_attention_mask(input).to(device)
        tgt_padding_mask = gen_attention_mask(output).to(device)

        predictions = model(input, output, src_mask, tgt_mask, src_padding_mask, tgt_padding_mask).transpose(0,1)
        # 현재(마지막) 시점의 예측 단어를 받아온다.
        predictions = predictions[:, -1:, :]
        predicted_id = torch.LongTensor(torch.argmax(predictions.cpu(), axis=-1))


        # 만약 마지막 시점의 예측 단어가 종료 토큰이라면 예측을 중단
        if torch.equal(predicted_id[0][0], torch.tensor(END_TOKEN[0])):
            break

        # 마지막 시점의 예측 단어를 출력에 연결한다.
        # 이는 for문을 통해서 디코더의 입력으로 사용될 예정이다.
        output = torch.cat([output, predicted_id.to(device)], axis=1)

    return torch.squeeze(output, axis=0).cpu().numpy()

def predict(sentence):
    prediction = evaluate(sentence)
    predicted_sentence = vocab.Decode(list(map(int,[i for i in prediction if i < vocab_size+7])))

    print('Input: {}'.format(sentence))
    print('Output: {}'.format(predicted_sentence))

    return predicted_sentence

In [39]:
model.load_state_dict(torch.load("chatbot.pth"))
result = predict("보험적용이 되나요?")

Input: 보험적용이 되나요?
Output: 고객님 이 경우 계약전환시간을 이해할 수 없는 경우라도 주의의무가 없는 경우 보험금 지급이 거절되거나 자동차 보험에 가입해야 하는 것입니다
